<a href="https://colab.research.google.com/github/KellyPared/deep-learning-challenge/blob/main/deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint



# 

In [2]:
#Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
#there is a fairly equal amount of is successful
application_df['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

Goal: "Wants a tool that can help it select the applicants for funding with the best chance of success.."

*   What variable(s) are the target(s) for your model?
*   What variable(s) are the feature(s) for your model?


### Target Variable is "Is Successful"
### The other variables(Except EIN and NAME) are the features.


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)



In [6]:
# calculate the number of unique values for each column
for col in application_df.columns:
    unique_vals = application_df[col].nunique()
    #print(f"{col}: {unique_vals}")
    print("{:<23}: {}".format(col, unique_vals))



APPLICATION_TYPE       : 17
AFFILIATION            : 6
CLASSIFICATION         : 71
USE_CASE               : 5
ORGANIZATION           : 4
STATUS                 : 2
INCOME_AMT             : 9
SPECIAL_CONSIDERATIONS : 2
ASK_AMT                : 8747
IS_SUCCESSFUL          : 2


In [7]:
# Look at APPLICATION_TYPE value counts for binning
# In ml, this column can be used as a feature to predict whether an organization will be successful in receiving funding.
app_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_counts)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [8]:
# calculate the number of unique values for each column
for col in application_df.columns:
    unique_vals = application_df[col].nunique()

    #For columns that have more than 10 unique values, determine the number of data points for each unique value.
    #anything else. just print
    
    if unique_vals > 10:
        print("Unique values for column '{}' (total: {}):".format(col, unique_vals))
        print(application_df[col].value_counts())
        print(" ")

    else:
        print("Unique Values less than 10")
        print("{:<23}: {}".format(col, unique_vals))
        print(" ")

Unique values for column 'APPLICATION_TYPE' (total: 17):
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
 
Unique Values less than 10
AFFILIATION            : 6
 
Unique values for column 'CLASSIFICATION' (total: 71):
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
 
Unique Values less than 10
USE_CASE               : 5
 
Unique Values less than 10
ORGANIZATION           : 4
 
Unique Values less than 10
STATUS                 : 2
 
Unique Values less than 10
INCOME_AMT             : 9
 
Unique Values less than 10
SPECIAL_CONSIDERATIONS : 2
 
Unique values for column 'ASK_AMT' (total: 8747):
5000        25398
10478     

There are 17 different T types of the application type; T-codes might correspond to different types of activities that the organization performs, such as training and technical assistance, consulting, research and evaluation, information dissemination


Choose a cutoff value that captures the majority of the variation in the data while still allowing you to bin the less common values into a single category.

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# The items below 500 are less frequent
#A good rule of thumb is to choose a cutoff value that reduces the number 
# of unique values in the column to a manageable number
cutoff = 500

# Get the counts of each application type
type_counts = application_df["APPLICATION_TYPE"].value_counts()

# application types less frequently than the cutoff
application_types_to_replace = list(type_counts[type_counts < cutoff].index)

# Print the types to be replaced
print(application_types_to_replace)


['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


In [10]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

This information is important because it can help us to decide how to handle infrequent values in the CLASSIFICATION column. One common approach is to bin the infrequent values together into a single category. This can help to reduce the dimensionality of the data and

In [11]:
# Look at CLASSIFICATION value counts for 
#use this information to determine which values occur frequently and which values occur infrequently

classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts_over_1 = classification_counts[classification_counts > 1]
print(classification_counts_over_1)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff2 = 300
# use the variable name `classifications_to_replace`
# Get the counts of each application type
class_counts = application_df['CLASSIFICATION'].value_counts()

# Identify the application types that occur less frequently than the cutoff
classifications_to_replace =  list(class_counts[class_counts < cutoff2].index)

classifications_to_replace 




['C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [14]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
# identify the categorical columns
cat_cols = application_df.select_dtypes(include=['object']).columns
cat_cols


Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
      dtype='object')

In [16]:
#OneHotEncoder
enc = OneHotEncoder(sparse=False)
encoded_df = pd.DataFrame(enc.fit_transform(application_df[cat_cols]))
encoded_df.columns = enc.get_feature_names_out()
encoded_df.head()

# convert the categorical columns to dummy variables
# final_df = pd.get_dummies(application_df, columns=cat_cols)

# apply one-hot encoding using pd.get_dummies
# final_df

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
from IPython.core import application
#merge one-hot encoder and drop originals

application_df = application_df.merge(encoded_df, left_index=True, right_index =True)
application_df = application_df.drop(cat_cols,1)
application_df.head()

<ipython-input-17-fba0bef732c8>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(cat_cols,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_C7000', 'CLASSIFICATION_Other',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_

In [19]:
#Split our preproesssed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values # remove the target column
y = application_df['IS_SUCCESSFUL'].values # set the target column as the target variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [20]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (25724, 44)
y_train shape: (25724,)
X_test shape: (8575, 44)
y_test shape: (8575,)


In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model
Create a neural network model by assigning the number of input features and nodes for each layer using TensorFlow and Keras.

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#Sequential = layers are stacked on top of each other in sequence
model = tf.keras.models.Sequential()

#the number of input features - this is the length of the training -pick any column
number_input_features = len(X_train_scaled[0])
print(number_input_features)

44


In [23]:

#hidden nodes for each layer pick a random number
units_layer1 = 30
units_layer2 = 20

#the more layers - 
# First hidden layer
model.add(Dense(units=units_layer1, 
                                activation='relu', 
                                input_dim=number_input_features))

# Second hidden layer
model.add(Dense(units=units_layer2, 
                                activation='relu', 
                                input_dim=number_input_features))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1350      
                                                                 
 dense_1 (Dense)             (None, 20)                620       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,991
Trainable params: 1,991
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model - 
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:

# Define the checkpoint file path
checkpoint_filepath = 'model_weights.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    period=5,
    verbose=1
)

weights = model.get_weights()
for i, layer_weights in enumerate(weights):
    print(f'Layer {i} weights:')
    print(layer_weights)


# Train the model
model.fit(X_train_scaled, y_train,epochs=100,
          validation_data=(X_test, y_test),
    callbacks=[checkpoint_callback])

Layer 0 weights:
[[ 0.04374507 -0.17091711  0.10754123 ... -0.05539891  0.11193019
  -0.02577055]
 [-0.08946584 -0.21575186  0.08470872 ... -0.09094821 -0.04773429
   0.07658347]
 [ 0.01443878  0.205762    0.02468875 ... -0.1482408  -0.15723203
   0.19214839]
 ...
 [ 0.1336382  -0.16182575  0.04738224 ... -0.07212114  0.23062292
   0.164487  ]
 [-0.26566207 -0.11542235 -0.01903778 ... -0.09049782  0.14805084
  -0.2746326 ]
 [-0.2510138   0.06152624  0.19570225 ...  0.0572089   0.10947552
   0.27385142]]
Layer 1 weights:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Layer 2 weights:
[[-0.18203087 -0.09047413  0.17967886 -0.16189784 -0.3412123   0.27086914
   0.25345713 -0.09195441 -0.17227817  0.07633379  0.28943288 -0.28334883
  -0.23110771 -0.09663738 -0.25142416 -0.0862084   0.14129883  0.11056265
  -0.15325655  0.19526976]
 [-0.2982206  -0.11418913  0.32626128 -0.08777624  0.17354387  0.11178607
  -0.12797771 -0.04384273  0.25113034  0.

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5543 - accuracy: 0.7280 - 224ms/epoch - 837us/step
Loss: 0.5543296933174133, Accuracy: 0.7280466556549072


In [27]:
# Export our model to HDF5 file
# save the model to an HDF5 file
model.save('AlphabetSoupCharity.h5')


# Optimizing the Model: Trial 2


In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#Sequential = layers are stacked on top of each other in sequence
model2 = tf.keras.models.Sequential()

#the number of input features - this is the length of the training -pick any column
number_input_features = len(X_train_scaled[0])
print(number_input_features)

44


In [34]:
#hidden nodes for each layer pick a random number
units_layer1 = 50
units_layer2 = 30

#the more layers - 
# First hidden layer
model2.add(Dense(units=units_layer1, 
                                activation='relu', 
                                input_dim=number_input_features))

# Second hidden layer
model2.add(Dense(units=units_layer2, 
                                activation='tanh', 
                                input_dim=number_input_features))

# Output layer
model2.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                2250      
                                                                 
 dense_4 (Dense)             (None, 30)                1530      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,811
Trainable params: 3,811
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model - 
model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Define the checkpoint file path
checkpoint_filepath = 'model_weights2.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    period=5,
    verbose=1
)

weights = model2.get_weights()
for i, layer_weights in enumerate(weights):
    print(f'Layer {i} weights:')
    print(layer_weights)


# Train the model
model2.fit(X_train_scaled, y_train,epochs=100,
          validation_data=(X_test, y_test),
    callbacks=[checkpoint_callback])

Layer 0 weights:
[[-0.09638971  0.0664556  -0.0693451  ... -0.06661662 -0.12065934
   0.20426849]
 [-0.07538232  0.15046117 -0.08218205 ...  0.19200003  0.03280437
  -0.09176357]
 [-0.02252659  0.2512479   0.16802925 ...  0.11660525  0.06333047
   0.12921938]
 ...
 [-0.23540597  0.19408798 -0.07421947 ...  0.24893305  0.15074253
  -0.22812459]
 [ 0.1515007  -0.15676022  0.07582065 ...  0.23942083  0.08367932
   0.20930314]
 [-0.07328697  0.0347845   0.01928809 ... -0.07080226 -0.20972635
   0.20275813]]
Layer 1 weights:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Layer 2 weights:
[[-0.11612764 -0.02251187 -0.15872951 ...  0.18456894  0.08648512
   0.20284867]
 [-0.10573068 -0.07613347 -0.06582995 ... -0.2118823  -0.05724867
  -0.15412977]
 [ 0.2147888  -0.25020018 -0.17367496 ... -0.0247487   0.11775497
  -0.2730487 ]
 ...
 [-0.1906782  -0.06678754  0.0563207  ...  0.18010658  

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7248 - 251ms/epoch - 936us/step
Loss: 0.5567113757133484, Accuracy: 0.724781334400177


In [40]:

# Export our model to HDF5 file
# save the model to an HDF5 file
model.save('AlphabetSoupCharity2.h5')

# Model3

In [55]:
#Import and read the charity_data.csv.
application_df2 = application_df.copy()


In [56]:


# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2.drop(['STATUS'], axis=1, inplace=True)


#Split our preproesssed data into our features and target arrays
X = application_df2.drop('IS_SUCCESSFUL', axis=1).values # remove the target column
y = application_df2['IS_SUCCESSFUL'].values # set the target column as the target variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

KeyError: ignored

In [53]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#Sequential = layers are stacked on top of each other in sequence
model3 = tf.keras.models.Sequential()

#the number of input features - this is the length of the training -pick any column
number_input_features = len(X_train_scaled[0])
print(number_input_features)

44


In [42]:
#hidden nodes for each layer pick a random number
units_layer1 = 30
units_layer2 = 20
units_layer3 = 10

#the more layers - 
# First hidden layer
model3.add(Dense(units=units_layer1, 
                                activation='relu', 
                                input_dim=number_input_features))

# Second hidden layer
model3.add(Dense(units=units_layer2, 
                                activation='tanh', 
                                input_dim=number_input_features))
# Third hidden layer
model3.add(Dense(units=units_layer3, 
                                activation='tanh', 
                                input_dim=number_input_features))

# Output layer
model3.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 30)                1350      
                                                                 
 dense_7 (Dense)             (None, 20)                620       
                                                                 
 dense_8 (Dense)             (None, 10)                210       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,191
Trainable params: 2,191
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model - 
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
# Define the checkpoint file path
checkpoint_filepath = 'model_weights2.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    period=5,
    verbose=1
)

weights = model3.get_weights()
for i, layer_weights in enumerate(weights):
    print(f'Layer {i} weights:')
    print(layer_weights)


# Train the model
model3.fit(X_train_scaled, y_train,epochs=50,
          validation_data=(X_test, y_test),
    callbacks=[checkpoint_callback])

Layer 0 weights:
[[ 0.24951968 -0.06057891 -0.02224922 ... -0.09999511 -0.15307193
  -0.06375477]
 [-0.1307388   0.11063507  0.15409079 ... -0.20852527 -0.19661069
  -0.13380508]
 [-0.0877105   0.03928646  0.253315   ... -0.04846716  0.25391087
  -0.10587707]
 ...
 [-0.1459414   0.18996319 -0.09153096 ... -0.06556448 -0.16311218
   0.10143259]
 [ 0.19011545  0.18695244 -0.15700419 ...  0.23566958  0.01670289
   0.12534901]
 [ 0.0336799   0.09189811  0.09694973 ... -0.13403386 -0.14397283
  -0.0638811 ]]
Layer 1 weights:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Layer 2 weights:
[[-0.30834827 -0.03932238 -0.22485931  0.1465463  -0.10598011  0.22486669
   0.19228029  0.02254534 -0.08201206  0.17646366  0.25701457  0.20493042
   0.329894   -0.28321883 -0.31782925  0.10279351  0.05490732  0.3417529
  -0.31361836 -0.02238643]
 [-0.18819033  0.23425078 -0.03612936 -0.25031507  0.2439056   0.2147817
   0.04804048  0.21064442  0.01490563  0.21

In [46]:

# Evaluate the model using the test data
model_loss, model_accuracy = model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5522 - accuracy: 0.7304 - 273ms/epoch - 1ms/step
Loss: 0.5522164106369019, Accuracy: 0.7303789854049683


In [47]:

# Export our model to HDF5 file
# save the model to an HDF5 file
model.save('AlphabetSoupCharity3.h5')
     

Depending on the model how much error can I tolerate-- look at the confusion matrix.

In [28]:
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


The accuracy of the model is 0.7266, which means it correctly predicts the outcome for 72.66% of the cases in the test set. While this is not a bad accuracy, whether this model is considered "good" or not depends on the specific context and requirements of the project.

For example, if the consequences of a false positive (predicting a success when it will actually fail) or a false negative (predicting a failure when it will actually succeed) are very different, then the accuracy alone may not be a sufficient metric to evaluate the model's performance.

It's important to also consider other metrics such as precision, recall, F1-score, and the confusion matrix to get a better understanding of the model's performance. Additionally, if the project requirements demand a higher accuracy, then further optimization may be necessary.

In [29]:
#look at the confusion matrix to determine if a 72% is a good 

from sklearn.metrics import confusion_matrix
y = application_df['IS_SUCCESSFUL']
X = application_df.drop('IS_SUCCESSFUL',axis=1)

# Check the balance of our target values
label_counts = y.value_counts()
label_counts


1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [30]:
# Make a prediction using the testing data
pred = model.predict(X_test)

268/268 [==============================] - 0s 898us/step


In [31]:
# predictions_df = pd.DataFrame({"Prediction": pred, "Actual": y_test})
# predictions_df.head()